In [63]:
import numpy as np
from scipy.optimize import linprog

def generate_bids(n, m, p_bar):
    """
    Generate a sequence of random bids.
    :param n: Total number of bids.
    :param m: Number of items.
    :param p_bar: Ground truth price vector.
    :return: Array of bids.
    """
    bids = []
    for _ in range(n):
        a_k = np.random.choice([0, 1], size=m)  # Generate a_k
        pi_k = np.dot(p_bar, a_k) + np.random.normal(0, np.sqrt(0.2))  # Calculate bid price
        bids.append((a_k, pi_k))
    return bids

def solve_offline_lp(bids, m, b_i):
    c = -np.array([pi_k for _, pi_k in bids])  # Negative for maximization
    A = np.array([a_k for a_k, _ in bids]).T  # Transpose to match dimensions
    b = b_i * np.ones(m)

    # Solving the LP
    result = linprog(c, A_ub=A, b_ub=b, bounds=(0, 1), method='highs')

    if result.success:
        return -result.fun  # Revenue (negate because of maximization)
    else:
        raise ValueError("Offline LP did not converge")

def solve_partial_lp_dual(bids, k, n, b_i):
    # Objective function: maximize sum(pi_j * x_j) for j=1 to k
    c = -np.array([pi_k for _, pi_k in bids[:k]])

    # Constraints: sum(a_ij * x_j) <= (k/n) * b_i for all i
    A = np.array([a_k for a_k, _ in bids[:k]]).T
    b = (k / n) * np.array(b_i)

    # Bounds for decision variables: 0 <= x_j <= 1
    x_bounds = [(0, 1) for _ in range(k)]

    # Solve the linear program
    result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

    # print(result.ineqlin.get('marginals'))

    if result.success:
        # The dual variable corresponding to the inequality constraints A_ub * x <= b_ub
        return result.get('slack'), -1*result.ineqlin.get('marginals'), result.get('fun')
    else:
        raise ValueError("failed to find a solution")

In [64]:
# Problem 1
def run_slpm_static(bids, k, n, b_i):
    revenue = 0

    # Solve the partial LP for the first k bids to get dual prices
    slack, dual_price, k_revenue = solve_partial_lp_dual(bids, k, n, b_i)
    revenue += -k_revenue
    remaining_capacity = np.array(b_i-((k/n)*b_i-slack))

    for i, (a_k, pi_k) in enumerate(bids):
        if i >= k:
            # Allocate based on the decision rule using y_bar
            if pi_k > np.dot(a_k, dual_price) and all(remaining_capacity - a_k >= 0):
                revenue += pi_k
                remaining_capacity -= a_k

    return revenue

# Problem 2
def run_slpm_dynamic(bids, k, n, b_i):
    revenue = 0

    # Solve the partial LP for the first k bids to get dual prices
    slack, dual_price, k_revenue = solve_partial_lp_dual(bids, k, n, b_i)
    revenue += -k_revenue
    remaining_capacity = np.array(b_i-((k/n)*b_i-slack))

    target = 2*k

    for i, (a_k, pi_k) in enumerate(bids):
        if i >= k:
            if i == target:
                slack, dual_price, _ = solve_partial_lp_dual(bids, target, n, b_i)
                print(dual_price)
                target *= 2
            # Allocate based on the decision rule using y_bar
            if pi_k > np.dot(a_k, dual_price) and all(remaining_capacity - a_k >= 0):
                revenue += pi_k
                remaining_capacity -= a_k

    return revenue

def solve_problem (bids_fixed, k_values, n, b_i, problem):
    print("Problem", problem)
    revenues = {}
    if problem == 1:
        for k in k_values:
            revenue = run_slpm_static(bids_fixed, k, n, b_i)
            revenues[k] = revenue

        for slpm_revenue, k in zip(revenues.values(), k_values):
            print(f"SLPM Static revenue: {slpm_revenue} at k={k}")
    elif problem == 2:
        revenue = run_slpm_dynamic(bids_fixed, 50, n, b_i)
        print(f"SLPM Dynamic revenue: {revenue}")

def run(n, m, p_bar, k_values):
    # Regenerate bids with the fixed p_bar
    bids_fixed = generate_bids(n, m, p_bar)

    b_i = np.ones(m) * 1000  # Bid cap for all i

    solve_problem(bids_fixed, k_values, n, b_i, 1)
    solve_problem(bids_fixed, k_values, n, b_i, 2)

    offline_revenue_value = solve_offline_lp(bids_fixed, m, b_i)

    print(f"Offline revenue: {offline_revenue_value}")

if __name__ == "__main__":
    n = 10000  # Total number of bids
    m = 10     # Number of items
    k_values = [50, 100, 200, 1000, 5000, 10000]  # Different k values to test

    # Fixed ground truth price vector (p_bar) - set to ones for simplicity
    price_vector = np.ones(m)  # Vector of ones

    run(n, m, price_vector, k_values)

Problem 1
SLPM Static revenue: 8477.303320970415 at k=50
SLPM Static revenue: 8741.081906941072 at k=100
SLPM Static revenue: 10267.762675880485 at k=200
SLPM Static revenue: 10412.553184980994 at k=1000
SLPM Static revenue: 11050.194150635158 at k=5000
SLPM Static revenue: 11288.916019687234 at k=10000
Problem 2
[1.03497279 0.8678694  1.02481271 1.23900682 0.76070782 1.18708055
 1.23171852 1.04231057 1.27457646 1.10021731]
[1.03497279 1.1479233  0.95881589 0.99383107 1.26361644 1.14941843
 1.01342649 0.96461115 0.9878182  1.01624729]
[1.10943162 1.09019855 1.11351183 1.00255938 1.18125621 1.05530542
 1.09857085 1.02194106 1.02377438 1.04286631]
[1.06780003 1.11044009 1.04635937 1.02650445 1.11154323 1.09537416
 1.10406469 1.05297861 1.12241227 1.00981081]
[1.04829114 1.08030616 1.01629471 1.06869213 1.12501586 1.11941911
 1.06564647 1.08456141 1.08980219 1.01543587]
[1.03188707 1.08001246 1.03914488 1.06510589 1.1063391  1.09992631
 1.0812025  1.08812736 1.10884026 1.02325258]
[1.0681